In [1]:
import pandas as pd
import scipy.stats as stats

# NHL Correlation
Calculating the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [2]:
def get_team(cities, df):
    splitted = df['team'].str.split()
    
    one_word = splitted.apply(lambda x: x[-1])
    df['team'][one_word.isin(cities['team'])] = one_word[one_word.isin(cities['team'])]
    
    two_words = splitted.apply(lambda x: ' '.join(x[-2:]))
    df['team'][two_words.isin(cities['team'])] = two_words[two_words.isin(cities['team'])]
    
    return df

In [3]:
def nhl_df_clean(cities):
    nhl_df = pd.read_csv("inputs/nhl.csv")
    
    nhl_df = nhl_df[nhl_df['year'] == 2018][['team','W','L']]
    nhl_df['team'] = nhl_df['team'].apply(lambda x: x.rstrip('*'))
    nhl_df = nhl_df.drop([0,9,18,26])
    
    nhl_df = get_team(cities, nhl_df)
    
    nhl_df = nhl_df.set_index('team').astype(float)
    nhl_df.loc['Rangers Islanders Devils'] = nhl_df.loc[['New York Rangers', 'New York Islanders', 'New Jersey Devils']].sum()
    nhl_df.loc['Kings Ducks'] = nhl_df.loc[['Los Angeles Kings', 'Anaheim Ducks']].sum()
    nhl_df = nhl_df.reset_index()
    
    nhl_df['W/L%'] = nhl_df[['W', 'L']].apply(lambda x: x['W'] / (x['W'] + x['L']), axis=1)
    
    return nhl_df

In [4]:
def cities_clean():
    cities = pd.read_html("inputs/wikipedia_data.html")[1]
    cities = cities.iloc[:-1,[0,3,5,6,7,8]]
    
    cities = cities.rename({'Population (2016 est.)[8]':'Population'}, axis=1)
    cities = cities.replace(['—', r'\[.*\]'], '', regex=True)
    
    return cities

In [5]:
cities = cities_clean()[['Metropolitan area','Population','NHL']]
cities = cities[cities['NHL'] != ''].rename({'NHL': 'team'}, axis=1)
cities.head()

,Metropolitan area,Population,team
0,New York City,20153634,Rangers Islanders Devils
1,Los Angeles,13310447,Kings Ducks
2,San Francisco Bay Area,6657982,Sharks
3,Chicago,9512999,Blackhawks
4,Dallas–Fort Worth,7233323,Stars


In [6]:
nhl_df = nhl_df_clean(cities)
nhl_df

,team,W,L,W/L%
0,Lightning,54.0,23.0,0.701299
1,Bruins,50.0,20.0,0.714286
2,Maple Leafs,49.0,26.0,0.653333
3,Panthers,44.0,30.0,0.594595
4,Red Wings,30.0,39.0,0.434783
5,Canadiens,29.0,40.0,0.420290
6,Senators,28.0,43.0,0.394366
7,Sabres,25.0,45.0,0.357143
8,Capitals,49.0,26.0,0.653333
9,Penguins,47.0,29.0,0.618421


In [7]:
teams = pd.merge(cities, nhl_df, how='left', on='team')
teams

,Metropolitan area,Population,team,W,L,W/L%
0,New York City,20153634,Rangers Islanders Devils,113.0,105.0,0.518349
1,Los Angeles,13310447,Kings Ducks,89.0,54.0,0.622378
2,San Francisco Bay Area,6657982,Sharks,45.0,27.0,0.625000
3,Chicago,9512999,Blackhawks,33.0,39.0,0.458333
4,Dallas–Fort Worth,7233323,Stars,42.0,32.0,0.567568
5,"Washington, D.C.",6131977,Capitals,49.0,26.0,0.653333
6,Philadelphia,6070500,Flyers,42.0,26.0,0.617647
7,Boston,4794447,Bruins,50.0,20.0,0.714286
8,Minneapolis–Saint Paul,3551036,Wild,45.0,26.0,0.633803
9,Denver,2853077,Avalanche,43.0,30.0,0.589041


In [8]:
pd.DataFrame([nhl_df['team'][~nhl_df['team'].isin(cities['team'])],cities['team']]).T

,team,team
12,New Jersey Devils,Red Wings
14,New York Islanders,NaN
15,New York Rangers,NaN
24,Anaheim Ducks,NaN
26,Los Angeles Kings,Predators
0,NaN,Rangers Islanders Devils
1,NaN,Kings Ducks
2,NaN,Sharks
3,NaN,Blackhawks
4,NaN,Stars


In [9]:
population_by_region = teams['Population'].astype(float)
win_loss_by_region = teams['W/L%']

stats.pearsonr(population_by_region, win_loss_by_region)[0]

np.float64(0.012308996455744282)

# NBA Correlation
Calculating the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [10]:
def nba_df_clean(cities):
    nba_df = pd.read_csv("inputs/nba.csv")
    
    nba_df = nba_df[nba_df['year'] == 2018][['team','W','L']]
    nba_df['team'] = nba_df['team'].replace('\*?\s\(.*\)', '', regex=True)
    
    nba_df = get_team(cities, nba_df)
    
    nba_df = nba_df.set_index('team').astype(float)
    nba_df.loc['Knicks Nets'] = nba_df.loc[['New York Knicks', 'Brooklyn Nets']].sum()
    nba_df.loc['Lakers Clippers'] = nba_df.loc[['Los Angeles Lakers', 'Los Angeles Clippers']].sum()
    nba_df = nba_df.reset_index()
    
    nba_df['W/L%'] = nba_df[['W', 'L']].apply(lambda x: x['W'] / (x['W'] + x['L']), axis=1)
    
    return nba_df

<>:5: SyntaxWarning: invalid escape sequence '\*'
<>:5: SyntaxWarning: invalid escape sequence '\*'
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_6596\970372192.py:5: SyntaxWarning: invalid escape sequence '\*'
  nba_df['team'] = nba_df['team'].replace('\*?\s\(.*\)', '', regex=True)


In [11]:
cities = cities_clean()[['Metropolitan area','Population','NBA']]
cities = cities[cities['NBA'] != ''].rename({'NBA': 'team'}, axis=1)
cities

,Metropolitan area,Population,team
0,New York City,20153634,Knicks Nets
1,Los Angeles,13310447,Lakers Clippers
2,San Francisco Bay Area,6657982,Warriors
3,Chicago,9512999,Bulls
4,Dallas–Fort Worth,7233323,Mavericks
5,"Washington, D.C.",6131977,Wizards
6,Philadelphia,6070500,76ers
7,Boston,4794447,Celtics
8,Minneapolis–Saint Paul,3551036,Timberwolves
9,Denver,2853077,Nuggets


In [12]:
nba_df = nba_df_clean(cities)
nba_df

,team,W,L,W/L%
0,Raptors,59.0,23.0,0.719512
1,Celtics,55.0,27.0,0.670732
2,76ers,52.0,30.0,0.634146
3,Cavaliers,50.0,32.0,0.609756
4,Pacers,48.0,34.0,0.585366
5,Heat,44.0,38.0,0.536585
6,Bucks,44.0,38.0,0.536585
7,Wizards,43.0,39.0,0.524390
8,Pistons,39.0,43.0,0.475610
9,Hornets,36.0,46.0,0.439024


In [13]:
teams = pd.merge(cities, nba_df, how='left', on='team')
teams

,Metropolitan area,Population,team,W,L,W/L%
0,New York City,20153634,Knicks Nets,57.0,107.0,0.347561
1,Los Angeles,13310447,Lakers Clippers,77.0,87.0,0.469512
2,San Francisco Bay Area,6657982,Warriors,58.0,24.0,0.707317
3,Chicago,9512999,Bulls,27.0,55.0,0.329268
4,Dallas–Fort Worth,7233323,Mavericks,24.0,58.0,0.292683
5,"Washington, D.C.",6131977,Wizards,43.0,39.0,0.524390
6,Philadelphia,6070500,76ers,52.0,30.0,0.634146
7,Boston,4794447,Celtics,55.0,27.0,0.670732
8,Minneapolis–Saint Paul,3551036,Timberwolves,47.0,35.0,0.573171
9,Denver,2853077,Nuggets,46.0,36.0,0.560976


In [14]:
pd.DataFrame([nba_df['team'][~nba_df['team'].isin(cities['team'])],cities['team']]).T

,team,team
10,New York Knicks,Heat
11,Brooklyn Nets,Suns
24,Los Angeles Clippers,Hornets
25,Los Angeles Lakers,Pacers
0,NaN,Knicks Nets
1,NaN,Lakers Clippers
2,NaN,Warriors
3,NaN,Bulls
4,NaN,Mavericks
5,NaN,Wizards


In [15]:
population_by_region = teams['Population'].astype(float)
win_loss_by_region = teams['W/L%']

stats.pearsonr(population_by_region, win_loss_by_region)[0]

np.float64(-0.17657160252844611)

# MLB Correlation
Calculating the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [16]:
def mlb_df_clean(cities):
    mlb_df = pd.read_csv("inputs/mlb.csv")
    
    mlb_df = mlb_df[mlb_df['year'] == 2018][['team','W','L']]
    
    mlb_df = get_team(cities, mlb_df)
    
    mlb_df = mlb_df.set_index('team').astype(float)
    mlb_df.loc['Dodgers Angels'] = mlb_df.loc[['Los Angeles Dodgers', 'Los Angeles Angels']].sum()
    mlb_df.loc['Yankees Mets'] = mlb_df.loc[['New York Yankees', 'New York Mets']].sum()
    mlb_df.loc['Giants Athletics'] = mlb_df.loc[['San Francisco Giants', 'Oakland Athletics']].sum()
    mlb_df.loc['Cubs White Sox'] = mlb_df.loc[['Chicago Cubs', 'Chicago White Sox']].sum()
    mlb_df = mlb_df.reset_index()
    
    mlb_df['W/L%'] = mlb_df[['W', 'L']].apply(lambda x: x['W'] / (x['W'] + x['L']), axis=1)
    
    return mlb_df

In [17]:
cities = cities_clean()[['Metropolitan area','Population','MLB']]
cities = cities[cities['MLB'] != ''].rename({'MLB': 'team'}, axis=1)
cities

,Metropolitan area,Population,team
0,New York City,20153634,Yankees Mets
1,Los Angeles,13310447,Dodgers Angels
2,San Francisco Bay Area,6657982,Giants Athletics
3,Chicago,9512999,Cubs White Sox
4,Dallas–Fort Worth,7233323,Rangers
5,"Washington, D.C.",6131977,Nationals
6,Philadelphia,6070500,Phillies
7,Boston,4794447,Red Sox
8,Minneapolis–Saint Paul,3551036,Twins
9,Denver,2853077,Rockies


In [18]:
mlb_df = mlb_df_clean(cities)
mlb_df

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_6596\2582425794.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['team'][two_words.isin(cities['team'])] = two_words[two_words.isin(cities['team'])]


,team,W,L,W/L%
0,Red Sox,108.0,54.0,0.666667
1,New York Yankees,100.0,62.0,0.617284
2,Rays,90.0,72.0,0.555556
3,Blue Jays,73.0,89.0,0.450617
4,Orioles,47.0,115.0,0.290123
5,Indians,91.0,71.0,0.561728
6,Twins,78.0,84.0,0.481481
7,Tigers,64.0,98.0,0.395062
8,Chicago White Sox,62.0,100.0,0.382716
9,Royals,58.0,104.0,0.358025


In [19]:
teams = pd.merge(cities, mlb_df, how='left', on='team')
teams

,Metropolitan area,Population,team,W,L,W/L%
0,New York City,20153634,Yankees Mets,177.0,147.0,0.546296
1,Los Angeles,13310447,Dodgers Angels,172.0,153.0,0.529231
2,San Francisco Bay Area,6657982,Giants Athletics,170.0,154.0,0.524691
3,Chicago,9512999,Cubs White Sox,157.0,168.0,0.483077
4,Dallas–Fort Worth,7233323,Rangers,67.0,95.0,0.413580
5,"Washington, D.C.",6131977,Nationals,82.0,80.0,0.506173
6,Philadelphia,6070500,Phillies,80.0,82.0,0.493827
7,Boston,4794447,Red Sox,108.0,54.0,0.666667
8,Minneapolis–Saint Paul,3551036,Twins,78.0,84.0,0.481481
9,Denver,2853077,Rockies,91.0,72.0,0.558282


In [20]:
pd.DataFrame([mlb_df['team'][~mlb_df['team'].isin(cities['team'])],cities['team']]).T

,team,team
1,New York Yankees,Dodgers Angels
8,Chicago White Sox,Twins
11,Oakland Athletics,Diamondbacks
13,Los Angeles Angels,Blue Jays
18,New York Mets,Indians
21,Chicago Cubs,Royals
25,Los Angeles Dodgers,NaN
28,San Francisco Giants,NaN
0,NaN,Yankees Mets
2,NaN,Giants Athletics


In [21]:
population_by_region = teams['Population'].astype(float)
win_loss_by_region = teams['W/L%']

stats.pearsonr(population_by_region, win_loss_by_region)[0]

np.float64(0.15052304487104853)

# NFL Correlation
Calculating the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [22]:
def nfl_df_clean(cities):
    nfl_df = pd.read_csv("inputs/nfl.csv")
    
    nfl_df = nfl_df[nfl_df['year'] == 2018][['team','W','L']]
    nfl_df['team'] = nfl_df['team'].apply(lambda x: x.rstrip('*+'))
    nfl_df = nfl_df.drop(range(0,36,5))
    
    nfl_df = get_team(cities, nfl_df)
    
    nfl_df = nfl_df.set_index('team').astype(float)
    nfl_df.loc['Giants Jets'] = nfl_df.loc[['New York Giants', 'New York Jets']].sum()
    nfl_df.loc['Rams Chargers'] = nfl_df.loc[['Los Angeles Rams', 'Los Angeles Chargers']].sum()
    nfl_df.loc['49ers Raiders'] = nfl_df.loc[['San Francisco 49ers', 'Oakland Raiders']].sum()
    nfl_df = nfl_df.reset_index()
    
    nfl_df['W/L%'] = nfl_df[['W', 'L']].apply(lambda x: x['W'] / (x['W'] + x['L']), axis=1)
    
    return nfl_df

In [23]:
cities = cities_clean()[['Metropolitan area','Population','NFL']]
cities['NFL'] = cities['NFL'].str.strip()
cities = cities[cities['NFL'] != ''].rename({'NFL': 'team'}, axis=1)

cities

,Metropolitan area,Population,team
0,New York City,20153634,Giants Jets
1,Los Angeles,13310447,Rams Chargers
2,San Francisco Bay Area,6657982,49ers Raiders
3,Chicago,9512999,Bears
4,Dallas–Fort Worth,7233323,Cowboys
5,"Washington, D.C.",6131977,Redskins
6,Philadelphia,6070500,Eagles
7,Boston,4794447,Patriots
8,Minneapolis–Saint Paul,3551036,Vikings
9,Denver,2853077,Broncos


In [24]:
nfl_df = nfl_df_clean(cities)
nfl_df

,team,W,L,W/L%
0,Patriots,11.0,5.0,0.687500
1,Dolphins,7.0,9.0,0.437500
2,Bills,6.0,10.0,0.375000
3,New York Jets,4.0,12.0,0.250000
4,Ravens,10.0,6.0,0.625000
5,Steelers,9.0,6.0,0.600000
6,Browns,7.0,8.0,0.466667
7,Bengals,6.0,10.0,0.375000
8,Texans,11.0,5.0,0.687500
9,Colts,10.0,6.0,0.625000


In [25]:
teams = pd.merge(cities, nfl_df, how='left', on='team')
teams

,Metropolitan area,Population,team,W,L,W/L%
0,New York City,20153634,Giants Jets,9.0,23.0,0.281250
1,Los Angeles,13310447,Rams Chargers,25.0,7.0,0.781250
2,San Francisco Bay Area,6657982,49ers Raiders,8.0,24.0,0.250000
3,Chicago,9512999,Bears,12.0,4.0,0.750000
4,Dallas–Fort Worth,7233323,Cowboys,10.0,6.0,0.625000
5,"Washington, D.C.",6131977,Redskins,7.0,9.0,0.437500
6,Philadelphia,6070500,Eagles,9.0,7.0,0.562500
7,Boston,4794447,Patriots,11.0,5.0,0.687500
8,Minneapolis–Saint Paul,3551036,Vikings,8.0,7.0,0.533333
9,Denver,2853077,Broncos,6.0,10.0,0.375000


In [26]:
pd.DataFrame([nba_df['team'][~nba_df['team'].isin(cities['team'])],cities['team']]).T

,team,team
0,Raptors,Giants Jets
1,Celtics,Rams Chargers
2,76ers,49ers Raiders
3,Cavaliers,Bears
4,Pacers,Cowboys
5,Heat,Redskins
6,Bucks,Eagles
7,Wizards,Patriots
8,Pistons,Vikings
9,Hornets,Broncos


In [27]:
population_by_region = teams['Population'].astype(float)
win_loss_by_region = teams['W/L%']

stats.pearsonr(population_by_region, win_loss_by_region)[0]

np.float64(0.004922112149349442)

# P_Values
Exploring the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports** with a series of paired t-tests between all pairs of sports.

In [28]:
cities = cities_clean()
for df in mlb_df, nhl_df, nba_df, nfl_df: df.set_index('team', inplace=True)

In [29]:
cities['NFL'] = cities['NFL'].str.strip()
cities = cities.set_index('Metropolitan area').drop('Population', axis=1)
cities = cities.where(cities != '')
cities['MLB'] = cities['MLB'].apply(lambda x: mlb_df['W/L%'][x] if type(x) == str else x)
cities['NHL'] = cities['NHL'].apply(lambda x: nhl_df['W/L%'][x] if type(x) == str else x)
cities['NBA'] = cities['NBA'].apply(lambda x: nba_df['W/L%'][x] if type(x) == str else x)
cities['NFL'] = cities['NFL'].apply(lambda x: nfl_df['W/L%'][x] if type(x) == str else x)

cities

,NFL,MLB,NBA,NHL
Metropolitan area,,,,
New York City,0.281250,0.546296,0.347561,0.518349
Los Angeles,0.781250,0.529231,0.469512,0.622378
San Francisco Bay Area,0.250000,0.524691,0.707317,0.625000
Chicago,0.750000,0.483077,0.329268,0.458333
Dallas–Fort Worth,0.625000,0.413580,0.292683,0.567568
"Washington, D.C.",0.437500,0.506173,0.524390,0.653333
Philadelphia,0.562500,0.493827,0.634146,0.617647
Boston,0.687500,0.666667,0.670732,0.714286
Minneapolis–Saint Paul,0.533333,0.481481,0.573171,0.633803


In [30]:
sports = ['NFL', 'NBA', 'NHL', 'MLB']
notNan = lambda k, j: ~(cities[k].isna()|cities[j].isna())
p_values = pd.DataFrame({k:{j:stats.ttest_rel(cities[k][notNan(k, j)], cities[j][notNan(k, j)]).pvalue for j in sports} for k in sports}, index=sports)

p_values

,NFL,NBA,NHL,MLB
NFL,NaN,0.941792,0.030959,0.802384
NBA,0.941792,NaN,0.022316,0.951046
NHL,0.030959,0.022316,NaN,0.000712
MLB,0.802384,0.951046,0.000712,NaN
